# 测算第四季度，每个业务的出货。
- 思路，把出货的日期，设置为按季度显示。
- 然后，按业务区分。 
- 把每个业务的四季度任务，放进去。
- 

In [5]:
#
import pandas as pd 
import numpy as np

prof = pd.read_excel('/home/bobo/下载/20.1月毛利.xlsx',header=2).fillna('/')
prof = prof[['单据编号','单据日期','客户名称','销售价(本)','成本价(本)','基本数量','销售收入(本)']]
prof.head()

,单据编号,单据日期,客户名称,销售价(本),成本价(本),基本数量,销售收入(本)
0,热水返利专用,/,/,/,/,/,/
1,SS-2020-01-00051,2020-01-06 00:00:00,李锟-涪陵区白涛镇专卖店高勇13983330380,400,0,-1,-400
2,小计,/,/,/,/,-1,-400
3,/,/,/,/,/,/,/
4,万家乐燃气热水器JSQ24-12K3.1,/,/,/,/,/,/


In [2]:
# prof = pd.read_table('https://raw.githubusercontent.com/bog5d/Feng-chao/master/profit2019',header=2).fillna('/')
# 尝试一次性把多个字符替换为空值 
prof['单据类型'] = prof['单据类型'].replace(['往来销售','销售退货','代销结算'],np.nan,regex=True)

# 向前填充
prof = prof.fillna(method='pad')
# 去掉小计那些列
prof = prof.loc[~(prof['单据日期']=='/')]

# 只筛选产品，赠品那些不计入进来。
#prof =prof.loc[prof['单据类型'].str.contains('万家乐')]

prof = prof[['单据类型', '单据日期','客户名称', '数量', '销售价', '成本价',
       '销售收入',  '销售毛利', '销售毛利率']]

#基本实现数据清洗
prof.head()

KeyError: '单据类型'

In [38]:
#  测算经销商去年和今年出货同期
''' 
明年开始出货已经不光是热水器了，所以第一个问题是，要精准比对客户的销售增长的话，只能测算同期热水器的提货增长
第二点： 表格要足够简略，但是这么多客户，只能按照区域。 或者按照业务。
还可以先借用原来的经销商测算方式来改进'''

data = prof.copy()

data = data.set_index('单据日期')

# 输入要测算的周期
### s18 = ('2018-5':'2018-12')
s80 = '2018-5'
s81 = '2018-12'

s90 = '2019-1'
s91 = '2019-12'
### s19 = ('2019-5':'2019-12')
data8 = data[s80:s81].groupby('客户名称').sum().sort_values('销售收入',ascending=False)[['销售收入']]

data8.columns=[s80+'至'+s81+'销售']
data8 = data8.reset_index()

In [39]:
data9 = data[s90:s91].groupby('客户名称').sum().sort_values('销售收入',ascending=False)[['销售收入']]

data9.columns=[s90+'至'+s91+'销售']
data9 = data9.reset_index()
data9.head(50)

,客户名称,2019-1至2019-12销售
0,国美结算,3569357.00
1,苏宁结算,2211010.09
2,王海梦-江津区小官山专卖店聂小敏13648366797,1186495.00
3,王海梦-石柱秦庭会18996922419,1032307.00
4,罗阳天-彭水县专卖店曾林周13594936789,692562.00
5,零散销售,606967.00
6,王海梦-永川区专卖店陶波18623611600,503465.00
7,王海梦-荣昌区王梅13996069608,460716.00
8,罗阳天-黔江区专卖店吴仕泉13996970436,432560.00
9,罗阳天-秀山县专卖店王俊凯18896183679,430037.00


In [40]:
data0 = pd.merge(data8,data9, how='left',on=['客户名称'])
data0 = data0.loc[data0['客户名称'].str.contains('-')]
data0

,2018-5至2018-12销售,客户名称,2019-1至2019-12销售


In [41]:
# 我们可以考虑来做对比。 
#  

In [42]:
s_1=s80+'至'+s81+'对比'+s90+'至'+s91
data0.to_csv(s_1)

In [43]:
%matplotlib inline
data0.plot()

TypeError: Empty 'DataFrame': no numeric data to plot

In [44]:
x = pd.concat([data8,data9],axis=1,join='inner')
x
#pd.merge(data8,data9,how='left')  #传出数组 

,客户名称,2018-5至2018-12销售,客户名称,2019-1至2019-12销售


In [45]:
# 测算客户销售并排名

df = prof.groupby('客户名称').sum().sort_values('销售毛利',ascending=False)[['数量','销售收入',  '销售毛利']]

df.loc['Row_sum'] = df.apply(lambda x: x.sum())

df  = df.reset_index()

df = df.loc[df['客户名称'].str.contains('-')]


df.reset_index(inplace=True,drop=True)
df.index = df.index+1
df

,客户名称,数量,销售收入,销售毛利
1,王海梦-江津区小官山专卖店聂小敏13648366797,2060.0,1186495.0,188630.52
2,王海梦-石柱秦庭会18996922419,865.0,1032307.0,152369.01
3,罗阳天-彭水县专卖店曾林周13594936789,1081.0,692562.0,147257.74
4,王海梦-荣昌区王梅13996069608,311.0,460716.0,91355.09
5,王海梦-主城-渝中区新华路批发商郑平13320203551,419.0,371570.0,80136.11
6,王海梦-永川区专卖店陶波18623611600,616.0,503465.0,78341.93
7,李锟-潼南批发李秀刚13350328937,417.0,306152.0,71390.14
8,李锟-铜梁柳朋15923286764,408.0,350921.0,69545.24
9,罗阳天-黔江区专卖店吴仕泉13996970436,405.0,432560.0,69281.78
10,罗阳天-秀山县专卖店王俊凯18896183679,431.0,430037.0,66048.25


In [46]:
prof['2020-1']

KeyError: '2020-1'

In [47]:
# 将业务和客户拆分开来。 

#
prof['业务'],prof['客户'] = prof['客户名称'].str.split('-',1).str



In [48]:
x = prof.loc[prof['业务'].str.contains('罗阳天')]
x.销售收入.sum()

2625648.0

In [49]:
prof['单据日期']= pd.to_datetime(prof['单据日期'])
prof = prof.set_index('单据日期')

In [50]:
data_q4 = prof['2019-12':'2019-12']
data_q4 = data_q4.groupby('业务').sum()[['销售收入']].nlargest(10,'销售收入')
data_q4

,销售收入
业务,
王海梦,504177.0
李锟,498834.0
罗阳天,376880.0
零散销售,80930.0
商渠,13800.0
大坪售后,9425.0
万进腾商贸有限公司,1620.0


In [28]:
data_q4['四季度目标任务']= 0


In [29]:

data_q4.loc['王海梦','四季度目标任务'] =2125000

data_q4.loc['李锟','四季度目标任务'] =1750000

data_q4.loc['罗阳天','四季度目标任务'] =1125000

data_q4['目标进度'] = round(data_q4['销售收入']/data_q4['四季度目标任务'],3)

data_q4.columns = ['Q4出货', 'Q4目标任务', '目标完成度']



data_q4['8折目标'] = data_q4['Q4目标任务']*0.8

data_q4['8折目标进度'] = round(data_q4['Q4出货']/data_q4['8折目标'],3)



## 把没有设定目标的行去除掉,这样就只剩王海梦，李锟，罗阳天
data_q4.loc[~(data_q4['Q4目标任务']==0)]


,Q4出货,Q4目标任务,目标完成度,8折目标,8折目标进度
业务,,,,,
王海梦,985812.0,2125000,0.464,1700000.0,0.580
李锟,671036.0,1750000,0.383,1400000.0,0.479
罗阳天,571161.0,1125000,0.508,900000.0,0.635


In [16]:
df2 = data_q4.loc[~(data_q4['Q4目标任务']==0)]
df2.to_csv('q4.csv')

# 分区域测算
- 思路，把每个客户的名称，前两个字提取出来，基本上就是地区名字， 
- 然后在groupby一下。 

In [17]:
prof1= prof['2019']
prof1['区域'] = prof1['客户'].str[0:2]

x = prof1.groupby('区域').sum()[['数量','销售收入']].nlargest(30,'销售收入')
## 可以把这些区县的社商排名 和我们的出货排名拿出来对比。 如果出货排名，远低于社商排名的，则属于困难区域。 
x#.to_csv('按区域汇总')


/home/bobo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,数量,销售收入
区域,,
江津,2098.0,1186670.0
主城,1478.0,965709.0
石柱,765.0,936307.0
黔江,751.0,782760.0
彭水,1085.0,695402.0
涪陵,621.0,605023.0
武隆,486.0,536942.0
潼南,621.0,525626.0
永川,616.0,503465.0


In [18]:
# 分区域做对比
x8 =  prof['2018-1':'2018-12']
x8['区域'] = x8['客户'].str[0:2]
x8 = x8.groupby('客户').sum()[['数量','销售收入']].nlargest(30,'销售收入')
x8= x8.reset_index()
x8.head(3)

/home/bobo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,客户,数量,销售收入
0,彭水县专卖店曾林周13594936789,438.0,536576.0
1,永川区专卖店陶波18623611600,669.0,383780.0
2,江津区小官山专卖店聂小敏13648366797,427.0,333602.0


In [19]:
# 分区域做对比
x9 =  prof['2019-1':'2019-12']
x9['区域'] = x9['客户'].str[0:2]
x9 = x9.groupby('客户').sum()[['数量','销售收入']].nlargest(30,'销售收入')
x9 = x9.reset_index()
x9.head()

/home/bobo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,客户,数量,销售收入
0,江津区小官山专卖店聂小敏13648366797,2060.0,1186495.0
1,石柱秦庭会18996922419,765.0,936307.0
2,彭水县专卖店曾林周13594936789,1081.0,692562.0
3,永川区专卖店陶波18623611600,616.0,503465.0
4,荣昌区王梅13996069608,311.0,460716.0


In [20]:
data_z = pd.merge(x8,x9,how='right',on=['客户'])
data_z.columns = ['区域', '18年数量', '18年销售收入', '19年数量', '19年销售收入']
data_z
#pd.merge(data8,data9, how='left',on=['客户名称'])

,区域,18年数量,18年销售收入,19年数量,19年销售收入
0,彭水县专卖店曾林周13594936789,438.0,536576.0,1081.0,692562.0
1,永川区专卖店陶波18623611600,669.0,383780.0,616.0,503465.0
2,江津区小官山专卖店聂小敏13648366797,427.0,333602.0,2060.0,1186495.0
3,璧山区专卖店丁应发18996039366,443.0,294183.0,603.0,339238.0
4,秀山县专卖店王俊凯18896183679,419.0,256870.0,431.0,430037.0
5,铜梁柳朋15923286764,406.0,235685.0,404.0,347301.0
6,合川专卖店李灵健13399879976,522.0,232300.0,257.0,270718.0
7,主城-渝中区新华路批发商郑平13320203551,239.0,222950.0,419.0,371570.0
8,綦江专卖店贺洪伟13594606456,165.0,202320.0,227.0,240498.0
9,荣昌区王梅13996069608,460.0,195440.0,311.0,460716.0


In [21]:
data_z.to_csv('客户对比')

In [22]:
# 按客户排名
prof.groupby('客户').sum()[['数量','销售收入']].nlargest(35,'销售收入')

,数量,销售收入
客户,,
江津区小官山专卖店聂小敏13648366797,2487.0,1520097.0
彭水县专卖店曾林周13594936789,1519.0,1229138.0
石柱秦庭会18996922419,765.0,936307.0
永川区专卖店陶波18623611600,1285.0,887245.0
秀山县专卖店王俊凯18896183679,850.0,686907.0
荣昌区王梅13996069608,771.0,656156.0
璧山区专卖店丁应发18996039366,1046.0,633421.0
主城-渝中区新华路批发商郑平13320203551,658.0,594520.0
黔江区专卖店吴仕泉13996970436,558.0,586630.0


### 流通型号测算
- 思路是先筛选，流通型号；去除老客户，剩下的则是新客户的提货
- 第二步，根据出货价格，算出应该给的提成金额。 
- 第三部，汇总给各业务。 （需明细表和汇总表）

In [23]:
data_circulate = prof.loc[prof['单据类型'].str.contains('F1')|prof['单据类型'].str.contains('GHF')]


In [24]:
# 老客户明细
x = ['李锟-合川专卖店-李灵健13399879976',
 '罗阳天-黔江区专卖店-吴仕泉13996970436',
 '李锟-涪陵区白涛镇专卖店-高勇13983330380',
 '李锟-潼南批发-李秀刚13350328937',
 '李锟-合川专卖店-李灵健13399879976',
 '商渠及其他客户',
 '罗阳天-酉阳县专卖店-黄忠平13996953429',
 '王海梦-璧山区专卖店-丁应发18996039366',
 '罗阳天-黔江-白江13308272849',
 '李锟-南川区水江镇专卖店-郑志刚15023505238',
 '罗阳天-武隆何梅17783326220',
 '罗阳天-酉阳县专卖店-黄忠平13996953429',
 '王海梦-江津区小官山专卖店-聂小敏13648366797',
 '王海梦-石柱秦庭会18996922419',
 '李锟-铜梁柳朋-15923286764',
 '王海梦-江津区小官山专卖店-聂小敏13648366797',
 '王海梦-大足龙腾建材市场专卖店-刘平18290408883',
 '李锟-涪陵专卖店-欧华13896660130',
 '王海梦-大足龙腾建材市场专卖店-刘平18290408883',
 '李锟-涪陵专卖店-欧华13896660130',
 '李锟-铜梁柳朋-15923286764',
 '李锟-南川-张显红13896616619',
 '零散销售',
 '罗阳天-武隆何梅17783326220',
 '罗阳天-武隆何梅17783326220',
 '国美结算',
 '王海梦-璧山区专卖店-丁应发18996039366',
 '商渠及其他客户',
 '国美结算',
 '罗阳天-武隆区专卖店-窦晓梅13228536396',
 '苏宁结算',
 '零散销售',
 '李锟-涪陵专卖店-欧华13896660130',
 '国美结算',
 '苏宁结算',
 '李锟-涪陵专卖店-欧华13896660130',
 '罗阳天-黔江-白江13308272849',
 '李锟-铜梁柳朋-15923286764',
 '国美结算',
 '李锟-潼南批发-李秀刚13350328937',
 '国美结算',
 '苏宁结算',
 '零散销售',
 '李锟-潼南批发-李秀刚13350328937',
 '王海梦-大足青少年宫万家乐专卖店-田飞燕13527589490',
 '国美结算',
 '王海梦-渝中区新华路批发商-郑平13320203551',
 '苏宁结算',
 '苏宁结算',
 '国美结算',
 '苏宁结算',
 '李锟-潼南批发-李秀刚13350328937',
 '王海梦-江津区小官山专卖店-聂小敏13648366797',
 '罗阳天-酉阳县专卖店-黄忠平13996953429',
 '国美结算',
 '王海梦-江津区小官山专卖店-聂小敏13648366797',
 '苏宁结算',
 '王海梦-璧山区专卖店-丁应发18996039366',
 '零散销售',
 '李锟-南川-张显红13896616619',
 '国美结算',
 '苏宁结算',
 '罗阳天-酉阳县专卖店-黄忠平13996953429',
 '零散销售',
 '国美结算',
 '苏宁结算',
 '李锟-铜梁柳朋-15923286764',
 '罗阳天-彭水县专卖店-曾林周13594936789',
 '国美结算',
 '国美结算',
 '国美结算',
 '王海梦-江津区小官山专卖店-聂小敏13648366797',
 '李锟-铜梁柳朋-15923286764',
 '国美结算',
 '李锟-铜梁柳朋-15923286764',
 '国美结算',
 '罗阳天-武隆区专卖店-窦晓梅13228536396',
 '苏宁结算',
 '李锟-綦江龙角路名气厨卫-冯丽15223208278',
 '国美结算',
 '国美结算',
 '国美结算',
 '国美结算',
 '王海梦-荣昌零售',
 '苏宁结算',
 '王海梦-大足龙腾建材市场专卖店-刘平18290408883',
 '罗阳天-酉阳县专卖店-黄忠平13996953429',
 '罗阳天-散货',
 '国美结算',
 '国美结算',
 '李锟-綦江龙角路名气厨卫-冯丽15223208278',
 '国美结算',
 '大坪售后',
 '大坪售后',
 '国美结算',
 '罗阳天-武隆区专卖店-窦晓梅13228536396',
 '王海梦-鱼洞中干道专卖店-杜贞兰15723344238',
 '商渠及其他客户',
 '国美结算',
 '苏宁结算',
 '国美结算',
 '罗阳天-黔江-白江13308272849',
 '苏宁结算',
 '国美结算',
 '苏宁结算',
 '李锟-潼南批发-李秀刚13350328937',
 '李锟-南川-张显红13896616619',
 '国美结算',
 '商渠及其他客户',
 '苏宁结算',
 '罗阳天-酉阳县专卖店-黄忠平13996953429',
 '国美结算',
 '国美结算',
 '罗阳天-秀山县专卖店-王俊凯18896183679',
 '零散销售',
 '王海梦-大足双桥-王波13509427312',
 '李锟-万盛区专卖店-傅建春13370751538',
 '罗阳天-武隆区专卖店-窦晓梅13228536396',
 '李锟-綦江专卖店-贺洪伟13594606456',
 '王海梦-主城九区散货',
 '罗阳天-黔江区专卖店-吴仕泉13996970436',
 '王海梦-大足区龙水镇专卖店-赵春13896155668',
 '商渠及其他客户',
 '国美结算',
 '罗阳天-黔江-白江13308272849',
 '王海梦-荣昌区-王梅13996069608',
 '罗阳天-黔江-白江13308272849',
 '王海梦-江津区小官山专卖店-聂小敏13648366797',
 '王海梦-鱼洞中干道专卖店-杜贞兰15723344238',
 '李锟-涪陵区白涛镇专卖店-高勇13983330380',
 '王海梦-白市驿专卖店-李政15213145597',
 '王海梦-渝中区新华路批发商-郑平13320203551',
 '李锟-潼南专卖店-贺健全13983615883',
 '罗阳天-秀山县专卖店-王俊凯18896183679',
 '王海梦-大足区龙水镇专卖店-赵春13896155668',
 '李锟-南川区水江镇专卖店-郑志刚15023505238',
 '王海梦-长生桥专卖店-张期文13696471299',
 '王海梦-荣昌区-王梅13996069608',
 '王海梦-大足双桥-王波13509427312',
 '王海梦-渝中区新华路批发商-郑平13320203551',
 '国美结算',
 '苏宁结算',
 '王海梦-石柱秦庭会18996922419',
 '王海梦-石柱秦庭会18996922419',
 '国美结算',
 '王海梦-主城九区散货',
 '王海梦-渝中区新华路批发商-郑平13320203551',
 '王海梦-主城九区散货',
 '李锟-潼南专卖店-贺健全13983615883',
 '罗阳天-彭水县专卖店-曾林周13594936789',
 '万进腾商贸有限公司',
 '万进腾商贸有限公司',
 '李锟-南川区水江镇专卖店-郑志刚15023505238',
 '苏宁结算',
 '李锟-綦江专卖店-贺洪伟13594606456',
 '王海梦-江津区小官山专卖店-聂小敏13648366797',
 '国美结算',
 '李锟-潼南批发-李秀刚13350328937',
 '王海梦-石柱秦庭会18996922419',
 '苏宁结算',
 '王海梦-主城九区散货',
 '万进腾商贸有限公司',
 '王海梦-大足青少年宫万家乐专卖店-田飞燕13527589490',
 '李锟-涪陵清溪镇-喻述琳13996813739',
 '国美结算',
 '罗阳天-秀山县专卖店-王俊凯18896183679',
 '罗阳天-武隆徐开福15736571888',
 '罗阳天-彭水县专卖店-曾林周13594936789',
 '李锟-长寿云集镇陶锦15223292529',
 '零散销售',
 '苏宁结算',
 '零散销售',
 '万进腾商贸有限公司',
 '万进腾商贸有限公司',
 '国美结算',
 '已售未提',
 '李锟-丰都高仕林-18581407666',
 '王海梦-石柱秦庭会18996922419',
 '罗阳天-黔江-白江13308272849',
 '罗阳天-彭水县专卖店-曾林周13594936789',
 '李锟-南川-张显红13896616619',
 '李锟-涪陵清溪镇-喻述琳13996813739',
 '苏宁结算',
 '李锟-潼南专卖店-贺健全13983615883',
 '王海梦-主城九区散货',
 '王海梦-大足双桥-王波13509427312',
 '罗阳天-黔江-白江13308272849',
 '王海梦-荣昌区荣隆镇专柜-夏兴国15823312458',
 '罗阳天-彭水县专卖店-曾林周13594936789',
 '王海梦-永川区专卖店-陶波18623611600',
 '国美结算',
 '王海梦-渝中区新华路批发商-郑平13320203551',
 '李锟-綦江区打通镇张云建13072384985',
 '王海梦-石柱秦庭会18996922419',
 '李锟-綦江区打通镇张云建13072384985',
 '罗阳天-黔江-白江13308272849',
 '苏宁结算',
 '国美结算',
 '罗阳天-黔江区专卖店-吴仕泉13996970436',
 '王海梦-主城九区散货',
 '王海梦-大足青少年宫万家乐专卖店-田飞燕13527589490',
 '罗阳天-武隆白马窦晓清15523664355',
 '国美结算',
 '罗阳天-黔江-白江13308272849',
 '王海梦-石柱秦庭会18996922419',
 '罗阳天-黔江-白江13308272849',
 '李锟-南川县城专卖店-杨淳之13609463539',
 '李锟-涪陵专卖店-欧华13896660130',
 '罗阳天-武隆白马窦晓清15523664355',
 '王海梦-大足区龙水镇专卖店-赵春13896155668',
 '王海梦-石柱秦庭会18996922419',
 '零散销售',
 '罗阳天-彭水县专卖店-曾林周13594936789',
 '罗阳天-黔江-白江13308272849',
 '李锟-涪陵区白涛镇专卖店-高勇13983330380',
 '罗阳天-武隆区专卖店-窦晓梅13228536396',
 '王海梦-江津区小官山专卖店-聂小敏13648366797',
 '罗阳天-武隆白马窦晓清15523664355',
 '罗阳天-彭水县专卖店-曾林周13594936789',
 '李锟-散货',
 '万进腾商贸有限公司',
 '国美结算',
 '罗阳天-秀山县专卖店-王俊凯18896183679',
 '国美结算',
 '苏宁结算',
 '零散销售',
 '李锟-长寿美大集成灶-雷相芳13114045376',
 '国美结算',
 '李锟-铜梁柳朋-15923286764',
 '李锟-合川专卖店-李灵健13399879976',
 '王海梦-江津区小官山专卖店-聂小敏13648366797',
 '苏宁结算',
 '李锟-合川专卖店-李灵健13399879976',
 '李锟-铜梁柳朋-15923286764',
 '国美结算',
 '李锟-潼南批发-李秀刚13350328937',
 '李锟-合川专卖店-李灵健13399879976',
 '苏宁结算',
 '国美结算',
 '零散销售',
 '李锟-长寿云集镇陶锦15223292529',
 '国美结算',
 '零散销售',
 '李锟-铜梁柳朋-15923286764',
 '李锟-潼南批发-李秀刚13350328937',
 '国美结算',
 '国美结算',
 '李锟-铜梁柳朋-15923286764',
 '国美结算',
 '苏宁结算',
 '李锟-铜梁柳朋-15923286764',
 '零散销售',
 '王海梦-界石曙光工业园-赵相虎13808301493',
 '李锟-铜梁柳朋-15923286764',
 '李锟-綦江专卖店-贺洪伟13594606456',
 '李锟-铜梁柳朋-15923286764',
 '李锟-綦江专卖店-贺洪伟13594606456',
 '零散销售',
 '李锟-铜梁柳朋-15923286764',
 '罗阳天-彭水县专卖店-曾林周13594936789',
 '李锟-铜梁柳朋-15923286764',
 '王海梦-大足青少年宫万家乐专卖店-田飞燕13527589490',
 '李锟-丰都高仕林-18581407666',
 '李锟-铜梁柳朋-15923286764',
 '罗阳天-秀山县专卖店-王俊凯18896183679',
 '罗阳天-秀山县专卖店-王俊凯18896183679',
 '王海梦-渝中区新华路批发商-郑平13320203551',
 '罗阳天-秀山县专卖店-王俊凯18896183679',
 '罗阳天-武隆区专卖店-窦晓梅13228536396',
 '王海梦-大足青少年宫万家乐专卖店-田飞燕13527589490',
 '万进腾商贸有限公司',
 '零散销售',
 '万进腾商贸有限公司',
 '李锟-铜梁柳朋-15923286764',
 '李锟-潼南批发-李秀刚13350328937',
 '李锟-铜梁柳朋-15923286764',
 '王海梦-大足青少年宫万家乐专卖店-田飞燕13527589490',
 '王海梦-江津区小官山专卖店-聂小敏13648366797',
 '王海梦-江津区小官山专卖店-聂小敏13648366797',
 '王海梦-荣昌区荣隆镇专柜-夏兴国15823312458',
 '罗阳天-秀山县专卖店-王俊凯18896183679',
 '罗阳天-黔江区专卖店-吴仕泉13996970436',
 '罗阳天-酉阳县专卖店-黄忠平13996953429',
 '罗阳天-彭水县专卖店-曾林周13594936789',
 '万进腾商贸有限公司',
 '罗阳天-武隆区专卖店-窦晓梅13228536396',
 '王海梦-大足青少年宫万家乐专卖店-田飞燕13527589490',
 '王海梦-江津区小官山专卖店-聂小敏13648366797',
 '王海梦-江津区小官山专卖店-聂小敏13648366797',
 '王海梦-荣昌区荣隆镇专柜-夏兴国15823312458',
 '罗阳天-彭水县专卖店-曾林周13594936789',
 '王海梦-江津区白沙镇专卖店-辜世兵18223187788',
 '万进腾商贸有限公司',
 '罗阳天-黔江-白江13308272849',
 '李锟-长寿云集镇陶锦15223292529',
 '王海梦-江津区小官山专卖店-聂小敏13648366797',
 '罗阳天-黔江区专卖店-吴仕泉13996970436',
 '李锟-涪陵区白涛镇专卖店-高勇13983330380',
 '李锟-潼南批发-李秀刚13350328937',
 '王海梦-鱼洞中干道专卖店-杜贞兰15723344238',
 '罗阳天-酉阳县专卖店-黄忠平13996953429',
 '王海梦-江津区小官山专卖店-聂小敏13648366797',
 '王海梦-石柱秦庭会18996922419',
 '王海梦-江津区小官山专卖店-聂小敏13648366797',
 '李锟-长寿桃花大道-余小群15123037348',
 '罗阳天-黔江区专卖店-吴仕泉13996970436',
 '李锟-涪陵专卖店-欧华13896660130',
 '王海梦-大足龙腾建材市场专卖店-刘平18290408883',
 '王海梦-大足双桥-王波13509427312',
 '王海梦-渝中区新华路批发商-郑平13320203551',
 '国美结算',
 '罗阳天-彭水县-刘小文15023985889',
 '王海梦-永川区专卖店-陶波18623611600',
 '王海梦-白市驿专卖店-李政15213145597',
 '王海梦-江津区小官山专卖店-聂小敏13648366797',
 '李锟-涪陵专卖店-欧华13896660130',
 '王海梦-渝中区新华路批发商-郑平13320203551',
 '李锟-涪陵区白涛镇专卖店-高勇13983330380',
 '王海梦-大足区龙水镇专卖店-赵春13896155668',
 '王海梦-大足龙水-伍敏15823806199',
 '王海梦-永川区专卖店-陶波18623611600',
 '王海梦-大足龙腾建材市场专卖店-刘平18290408883',
 '王海梦-江津区小官山专卖店-聂小敏13648366797',
 '国美结算',
 '苏宁结算',
 '苏宁结算',
 '王海梦-大足龙水-伍敏15823806199',
 '国美结算',
 '苏宁结算',
 '国美结算',
 '苏宁结算',
 '国美结算',
 '国美结算',
 '苏宁结算',
 '国美结算',
 '国美结算',
 '国美结算',
 '苏宁结算',
 '国美结算',
 '王海梦-永川区专卖店-陶波18623611600',
 '王海梦-大足龙腾建材市场专卖店-刘平18290408883',
 '罗阳天-武隆徐开福15736571888',
 '王海梦-永川区专卖店-陶波18623611600',
 '王海梦-大足龙水-伍敏15823806199',
 '苏宁结算',
 '王海梦-大足青少年宫万家乐专卖店-田飞燕13527589490',
 '大坪售后',
 '零散销售',
 '江八县售后',
 '国美结算',
 '苏宁结算',
 '李锟-铜梁柳朋-15923286764',
 '罗阳天-黔江-白江13308272849',
 '苏宁结算',
 '李锟-潼南批发-李秀刚13350328937',
 '李锟-丰都高仕林-18581407666',
 '苏宁结算',
 '李锟-铜梁柳朋-15923286764',
 '李锟-涪陵清溪镇-喻述琳13996813739',
 '李锟-长寿云集镇陶锦15223292529',
 '苏宁结算',
 '苏宁结算',
 '国美结算',
 '国美结算',
 '苏宁结算',
 '李锟-铜梁柳朋-15923286764',
 '苏宁结算',
 '王海梦-大足龙水-伍敏15823806199',
 '罗阳天-黔江-白江13308272849',
 '苏宁结算',
 '苏宁结算',
 '王海梦-大足龙腾建材市场专卖店-刘平18290408883',
 '王海梦-江津区小官山专卖店-聂小敏13648366797',
 '李锟-万盛区专卖店-傅建春13370751538',
 '王海梦-江津区小官山专卖店-聂小敏13648366797',
 '罗阳天-彭水县专卖店-曾林周13594936789',
 '王海梦-江津区小官山专卖店-聂小敏13648366797',
 '苏宁结算',
 '李锟-铜梁柳朋-15923286764',
 '李锟-涪陵马鞍镇专柜-刘成江13996771478',
 '万进腾商贸有限公司',
 '李锟-潼南批发-李秀刚13350328937',
 '王海梦-大足青少年宫万家乐专卖店-田飞燕13527589490',
 '李锟-涪陵清溪镇-喻述琳13996813739',
 '国美结算',
 '罗阳天-彭水县专卖店-曾林周13594936789',
 '王海梦-巴南区华南城-牟华13308336922',
 '大坪售后',
 '李锟-万盛区专卖店-傅建春13370751538',
 '李锟-万盛区专卖店-傅建春13370751538',
 '国美结算',
 '李锟-丰都高仕林-18581407666',
 '罗阳天-黔江区专卖店-吴仕泉13996970436',
 '王海梦-荣昌区荣隆镇专柜-夏兴国15823312458',
 '王海梦-巴南区华南城-牟华13308336922',
 '已售未提',
 '李锟-合川专卖店-李灵健13399879976',
 '零散销售',
 '大坪售后',
 '零散销售',
 '零散销售',
 '王海梦-主城九区散货',
 '李锟-万盛区专卖店-傅建春13370751538',
 '李锟-南川区水江镇专卖店-郑志刚15023505238',
 '万进腾商贸有限公司',
 '零散销售',
 '李锟-涪陵专卖店-欧华13896660130',
 '李锟-丰都县专卖店-苏正勇17320389351',
 '王海梦-荣昌零售',
 '王海梦-主城九区散货',
 '商渠及其他客户',
 '零散销售',
 '零散销售',
 '国美结算',
 '王海梦-江津区小官山专卖店-聂小敏13648366797',
 '王海梦-鱼洞中干道专卖店-杜贞兰15723344238',
 '王海梦-大足龙腾建材市场专卖店-刘平18290408883',
 '李锟-长寿帅邦-黄小波13102369338',
 '罗阳天-武隆徐开福15736571888',
 '罗阳天-武隆区专卖店-窦晓梅13228536396',
 '李锟-丰都县专卖店-苏正勇17320389351',
 '李锟-涪陵马鞍镇专柜-刘成江13996771478',
 '李锟-涪陵区白涛镇专卖店-高勇13983330380',
 '王海梦-九龙坡区曾家镇专卖店-唐勇13983220787',
 '王海梦-主城九区散货',
 '王海梦-大足龙腾建材市场专卖店-刘平18290408883',
 '王海梦-白市驿专卖店-李政15213145597',
 '王海梦-白市驿专卖店-李政15213145597',
 '王海梦-大足龙水-伍敏15823806199',
 '王海梦-主城九区散货',
 '王海梦-大足万古-欧丽梅13308326062',
 '罗阳天-秀山县专卖店-王俊凯18896183679',
 '王海梦-渝中区新华路批发商-郑平13320203551',
 '李锟-南川区水江镇专卖店-郑志刚15023505238',
 '万进腾商贸有限公司',
 '王海梦-大足区龙水镇专卖店-赵春13896155668',
 '李锟-涪陵荣事达厨卫-梁永刚13638228227',
 '王海梦-永川区专卖店-陶波18623611600',
 '李锟-散货',
 '李锟-散货',
 '王海梦-荣昌区-王梅13996069608',
 '李锟-綦江九龙大道帅康厨卫-胡小松15823568881',
 '罗阳天-黔江-白江13308272849',
 '零散销售',
 '王海梦-永川区专卖店-陶波18623611600',
 '零散销售',
 '李锟-綦江专卖店-贺洪伟13594606456',
 '李锟-长寿-陈波13983364973',
 '王海梦-九龙坡区西彭镇专卖店-徐老板13883422468',
 '王海梦-主城九区散货',
 '王海梦-璧山区专卖店-丁应发18996039366',
 '国美结算',
 '零散销售',
 '李锟-涪陵专卖店-欧华13896660130',
 '苏宁结算',
 '零散销售',
 '李锟-丰都高仕林-18581407666',
 '李锟-涪陵专卖店-欧华13896660130',
 '王海梦-石柱秦庭会18996922419',
 '王海梦-长生桥专卖店-张期文13696471299',
 '王海梦-大足双桥-王波13509427312',
 '罗阳天-黔江-白江13308272849',
 '李锟-万盛区专卖店-傅建春13370751538',
 '国美结算',
 '零散销售',
 '王海梦-永川区专卖店-陶波18623611600',
 '万进腾商贸有限公司',
 '万进腾商贸有限公司',
 '李锟-铜梁柳朋-15923286764',
 '王海梦-永川区专卖店-陶波18623611600',
 '李锟-潼南专卖店-贺健全13983615883',
 '零散销售',
 '罗阳天-彭水县专卖店-曾林周13594936789',
 '李锟-铜梁柳朋-15923286764',
 '罗阳天-黔江-白江13308272849',
 '李锟-南川县城专卖店-杨淳之13609463539',
 '王海梦-九龙坡区西彭镇专卖店-徐老板13883422468',
 '苏宁结算',
 '王海梦-主城九区散货',
 '王海梦-江津区小官山专卖店-聂小敏13648366797',
 '王海梦-江津区小官山专卖店-聂小敏13648366797',
 '王海梦-主城九区散货',
 '王海梦-巴南区华南城-牟华13308336922',
 '王海梦-永川区专卖店-陶波18623611600',
 '王海梦-大足龙水-伍敏15823806199',
 '李锟-铜梁柳朋-15923286764',
 '王海梦-渝北区回兴街道专卖店-张皓月15523325256',
 '国美结算',
 '王海梦-大足龙水-伍敏15823806199',
 '李锟-潼南批发-李秀刚13350328937',
 '零散销售',
 '王海梦-主城九区散货',
 '王海梦-永川区专卖店-陶波18623611600',
 '万进腾商贸有限公司',
 '零散销售',
 '零散销售',
 '苏宁结算',
 '李锟-达州-余杰15182832850',
 '王海梦-石柱秦庭会18996922419',
 '王海梦-主城九区散货',
 '零散销售',
 '王海梦-主城九区散货',
 '王海梦-渝北区回兴街道专卖店-张皓月15523325256',
 '李锟-铜梁柳朋-15923286764',
 '李锟-潼南批发-李秀刚13350328937',
 '万进腾商贸有限公司',
 '王海梦-璧山区专卖店-丁应发18996039366',
 '王海梦-大足青少年宫万家乐专卖店-田飞燕13527589490',
 '李锟-长寿桃花大道-余小群15123037348',
 '李锟-涪陵清溪镇-喻述琳13996813739',
 '李锟-綦江专卖店-贺洪伟13594606456',
 '国美结算',
 '李锟-散货',
 '罗阳天-彭水县专卖店-曾林周13594936789',
 '罗阳天-武隆徐开福15736571888',
 '罗阳天-黔江区专卖店-吴仕泉13996970436',
 '李锟-长寿-陈波13983364973',
 '零散销售',
 '罗阳天-武隆区专卖店-窦晓梅13228536396',
 '零散销售',
 '李锟-长寿云集镇陶锦15223292529',
 '李锟-丰都高仕林-18581407666',
 '零散销售',
 '王海梦-大足零售',
 '李锟-散货',
 '李锟-南川-张显红13896616619',
 '苏宁结算',
 '苏宁结算',
 '苏宁结算',
 '王海梦-渝北区回兴街道专卖店-张皓月15523325256',
 '王海梦-大足青少年宫万家乐专卖店-田飞燕13527589490',
 '罗阳天-黔江-白江13308272849',
 '李锟-散货',
 '李锟-南川县城专卖店-杨淳之13609463539',
 '李锟-綦江九龙大道帅康厨卫-胡小松15823568881',
 '王海梦-大足青少年宫万家乐专卖店-田飞燕13527589490',
 '王海梦-主城九区散货',
 '商渠及其他客户',
 '李锟-长寿美大集成灶-雷相芳13114045376',
 '国美结算',
 '国美结算',
 '王海梦-江津区小官山专卖店-聂小敏13648366797',
 '李锟-铜梁柳朋-15923286764',
 '王海梦-鱼洞中干道专卖店-杜贞兰15723344238',
 '罗阳天-武隆区专卖店-窦晓梅13228536396',
 '万进腾商贸有限公司',
 '李锟-綦江专卖店-贺洪伟13594606456',
 '李锟-涪陵专卖店-欧华13896660130',
 '王海梦-主城九区散货',
 '零散销售',
 '李锟-涪陵清溪镇-喻述琳13996813739',
 '李锟-南川-张显红13896616619',
 '苏宁结算',
 '李锟-长寿桃花大道-余小群15123037348',
 '李锟-涪陵马鞍镇专柜-刘成江13996771478',
 '李锟-潼南专卖店-贺健全13983615883',
 '王海梦-大足双桥-王波13509427312',
 '王海梦-大足青少年宫万家乐专卖店-田飞燕13527589490',
 '王海梦-荣昌区荣隆镇专柜-夏兴国15823312458',
 '李锟-铜梁柳朋-15923286764',
 '零散销售',
 '王海梦-大足龙腾建材市场专卖店-刘平18290408883',
 '李锟-铜梁柳朋-15923286764',
 '王海梦-荣昌区-王梅13996069608',
 '王海梦-主城九区散货',
 '王海梦-大足区龙水镇专卖店-赵春13896155668',
 '李锟-南川区水江镇专卖店-郑志刚15023505238',
 '王海梦-白市驿专卖店-李政15213145597',
 '国美结算',
 '李锟-潼南专卖店-贺健全13983615883',
 '李锟-綦江区打通镇张云建13072384985',
 '万进腾商贸有限公司',
 '李锟-綦江区打通镇张云建13072384985',
 '零散销售',
 '苏宁结算',
 '国美结算',
 '王海梦-主城九区散货',
 '王海梦-永川区专卖店-陶波18623611600',
 '零散销售',
 '万进腾商贸有限公司',
 '李锟-代华强-丰都周仕学18290328368',
 '李锟-丰都高仕林-18581407666',
 '零散销售',
 '万进腾商贸有限公司',
 '王海梦-主城九区散货',
 '国美结算',
 '李锟-丰都高仕林-18581407666',
 '王海梦-石柱秦庭会18996922419',
 '零散销售',
 '王海梦-石柱秦庭会18996922419',
 '苏宁结算',
 '苏宁结算',
 '零散销售',
 '罗阳天-武隆徐开福15736571888',
 '李锟-合川专卖店-李灵健13399879976',
 '王海梦-大足青少年宫万家乐专卖店-田飞燕13527589490',
 '罗阳天-武隆区专卖店-窦晓梅13228536396',
 '李锟-铜梁柳朋-15923286764',
 '罗阳天-秀山县专卖店-王俊凯18896183679',
 '王海梦-大足龙水-伍敏15823806199',
 '李锟-铜梁柳朋-15923286764',
 '王海梦-白市驿专卖店-李政15213145597',
 '李锟-涪陵区白涛镇专卖店-高勇13983330380',
 '李锟-合川专卖店-李灵健13399879976',
 '李锟-潼南批发-李秀刚13350328937',
 '国美结算',
 '王海梦-永川区专卖店-陶波18623611600',
 '李锟-长寿美大集成灶-雷相芳13114045376',
 '零散销售',
 '李锟-长寿帅邦-黄小波13102369338',
 '李锟-綦江九龙大道帅康厨卫-胡小松15823568881',
 '王海梦-九龙坡区曾家镇专卖店-唐勇13983220787',
 '李锟-綦江专卖店-贺洪伟13594606456',
 '王海梦-石柱秦庭会18996922419',
 '李锟-涪陵马鞍镇专柜-刘成江13996771478',
 '李锟-长寿-陈波13983364973',
 '李锟-潼南专卖店-贺健全13983615883',
 '零散销售',
 '李锟-南川区水江镇专卖店-郑志刚15023505238',
 '李锟-南川县城专卖店-杨淳之13609463539',
 '苏宁结算',
 '王海梦-永川区专卖店-陶波18623611600',
 '王海梦-界石曙光工业园-赵相虎13808301493',
 '李锟-丰都县专卖店-苏正勇17320389351',
 '李锟-涪陵专卖店-欧华13896660130',
 '王海梦-荣昌区-王梅13996069608',
 '罗阳天-散货',
 '李锟-涪陵专卖店-欧华13896660130',
 '王海梦-主城九区散货',
 '王海梦-江津区小官山专卖店-聂小敏13648366797',
 '李锟-潼南批发-李秀刚13350328937',
 '王海梦-渝北区回兴街道专卖店-张皓月15523325256',
 '李锟-丰都高仕林-18581407666',
 '王海梦-大足龙腾建材市场专卖店-刘平18290408883',
 '李锟-铜梁柳朋-15923286764',
 '罗阳天-彭水县专卖店-曾林周13594936789',
 '李锟-南川火星人陈忠祥13368044900',
 '李锟-潼南专卖店-贺健全13983615883',
 '王海梦-大足双桥-王波13509427312',
 '王海梦-江津区小官山专卖店-聂小敏13648366797',
 '罗阳天-武隆区专卖店-窦晓梅13228536396',
 '罗阳天-彭水县专卖店-曾林周13594936789',
 '李锟-长寿-陈波13983364973',
 '零散销售',
 '零散销售',
 '王海梦-永川区专卖店-陶波18623611600',
 '李锟-万盛区专卖店-傅建春13370751538',
 '商渠及其他客户',
 '李锟-铜梁柳朋-15923286764',
 '李锟-散货',
 '李锟-铜梁柳朋-15923286764',
 '李锟-铜梁柳朋-15923286764',
 '万进腾商贸有限公司',
 '王海梦-渝北区回兴街道专卖店-张皓月15523325256',
 '李锟-代华强-涪陵马鞍-张志朋13452553885',
 '万进腾商贸有限公司',
 '王海梦-江津区小官山专卖店-聂小敏13648366797',
 '王海梦-荣昌零售',
 '国美结算',
 '零散销售',
 '国美结算',
 '苏宁结算',
 '国美结算',
 '李锟-潼南专卖店-贺健全13983615883',
 '王海梦-江津区小官山专卖店-聂小敏13648366797',
 '零散销售',
 '李锟-南川火星人陈忠祥13368044900',
 '李锟-铜梁柳朋-15923286764',
 '李锟-长寿帅邦-黄小波13102369338',
 '苏宁结算',
 '李锟-散货',
 '国美结算',
 '李锟-潼南批发-李秀刚13350328937',
 '李锟-万盛区专卖店-傅建春13370751538',
 '李锟-合川专卖店-李灵健13399879976',
 '王海梦-白市驿专卖店-李政15213145597',
 '李锟-丰都高仕林-18581407666',
 '苏宁结算',
 '李锟-南川美大-王乾锋15826211989',
 '王海梦-大足青少年宫万家乐专卖店-田飞燕13527589490',
 '李锟-涪陵清溪镇-喻述琳13996813739',
 '国美结算',
 '万进腾商贸有限公司',
 '李锟-长寿-陈波13983364973',
 '李锟-南川美大-王乾锋15826211989',
 '李锟-长寿云集镇陶锦15223292529',
 '李锟-长寿帅邦-黄小波13102369338',
 '李锟-丰都高仕林-18581407666',
 '苏宁结算',
 '苏宁结算',
 '李锟-潼南批发-李秀刚13350328937',
 '李锟-涪陵清溪镇-喻述琳13996813739',
 '万进腾商贸有限公司',
 '零散销售',
 '万进腾商贸有限公司',
 '罗阳天-武隆白马窦晓清15523664355',
 '国美结算',
 '王海梦-江津区小官山专卖店-聂小敏13648366797',
 '罗阳天-彭水县专卖店-曾林周13594936789',
 '王海梦-九龙坡区西彭镇专卖店-徐老板13883422468',
 '王海梦-鱼洞中干道专卖店-杜贞兰15723344238',
 '王海梦-大足龙水-伍敏15823806199',
 '李锟-涪陵区白涛镇专卖店-高勇13983330380',
 '王海梦-石柱秦庭会18996922419',
 '罗阳天-武隆区专卖店-窦晓梅13228536396',
 '李锟-长寿云集镇陶锦15223292529',
 '罗阳天-武隆区专卖店-窦晓梅13228536396',
 '李锟-潼南批发-李秀刚13350328937',
 '罗阳天-酉阳县专卖店-黄忠平13996953429',
 '王海梦-渝北区回兴街道专卖店-张皓月15523325256',
 '零散销售',
 '罗阳天-秀山县专卖店-王俊凯18896183679',
 '李锟-丰都高仕林-18581407666',
 '李锟-南川-张显红13896616619',
 '罗阳天-武隆区专卖店-窦晓梅13228536396',
 '李锟-涪陵清溪镇-喻述琳13996813739',
 '李锟-合川专卖店-李灵健13399879976',
 '李锟-潼南批发-李秀刚13350328937',
 '苏宁结算',
 '李锟-长寿桃花大道-余小群15123037348',
 '李锟-潼南专卖店-贺健全13983615883',
 '李锟-涪陵马鞍镇专柜-刘成江13996771478',
 '罗阳天-黔江-白江13308272849',
 '国美结算',
 '苏宁结算',
 '国美结算',
 '国美结算',
 '苏宁结算',
 '国美结算',
 '大坪售后',
 '大坪售后',
 '李锟-万盛区专卖店-傅建春13370751538',
 '罗阳天-武隆区专卖店-窦晓梅13228536396',
 '零散销售',
 '商渠及其他客户',
 '商渠及其他客户',
 '商渠及其他客户',
 '国美结算',
 '王海梦-永川区专卖店-陶波18623611600',
 '李锟-散货',
 '王海梦-江津区小官山专卖店-聂小敏13648366797',
 '王海梦-白市驿专卖店-李政15213145597',
 '王海梦-大足龙腾建材市场专卖店-刘平18290408883',
 '李锟-丰都县专卖店-苏正勇17320389351',
 '王海梦-大足万古-欧丽梅13308326062',
 '李锟-潼南专卖店-贺健全13983615883',
 '李锟-南川区水江镇专卖店-郑志刚15023505238',
 '李锟-散货',
 '国美结算',
 '苏宁结算',
 '李锟-丰都高仕林-18581407666',
 '李锟-丰都县专卖店-苏正勇17320389351',
 '国美结算',
 '万进腾商贸有限公司',
 '王海梦-主城九区散货',
 '李锟-铜梁柳朋-15923286764',
 '罗阳天-黔江-白江13308272849',
 '罗阳天-秀山县专卖店-王俊凯18896183679',
 '零散销售',
 '李锟-南川区水江镇专卖店-郑志刚15023505238',
 '已售未提',
 '李锟-铜梁柳朋-15923286764',
 '万进腾商贸有限公司',
 '苏宁结算',
 '李锟-万盛区专卖店-傅建春13370751538',
 '零散销售',
 '李锟-南川专卖店-王俊利13908380118',
 '王海梦-石柱秦庭会18996922419',
 '万进腾商贸有限公司',
 '万进腾商贸有限公司',
 '国美结算',
 '李锟-散货',
 '王海梦-大足龙水-伍敏15823806199',
 '零散销售',
 '李锟-潼南批发-李秀刚13350328937',
 '李锟-合川专卖店-李灵健13399879976',
 '零散销售',
 '万进腾商贸有限公司',
 '李锟-散货',
 '李锟-长寿帅邦-黄小波13102369338',
 '苏宁结算',
 '苏宁结算',
 '王海梦-主城九区散货',
 '王海梦-主城九区散货',
 '零散销售',
 '零散销售',
 '王海梦-大足青少年宫万家乐专卖店-田飞燕13527589490',
 '李锟-涪陵清溪镇-喻述琳13996813739',
 '国美结算',
 '大坪售后',
 '王海梦-石柱秦庭会18996922419',
 '李锟-长寿帅邦-黄小波13102369338',
 '零散销售',
 '王海梦-大足双桥-王波13509427312',
 '苏宁结算',
 '苏宁结算',
 '李锟-潼南专卖店-贺健全13983615883',
 '李锟-万盛区专卖店-傅建春13370751538',
 '罗阳天-彭水县专卖店-曾林周13594936789',
 '国美结算',
 '王海梦-江津区小官山专卖店-聂小敏13648366797',
 '李锟-丰都高仕林-18581407666',
 '零散销售',
 '李锟-涪陵专卖店-欧华13896660130',
 '罗阳天-黔江区专卖店-吴仕泉13996970436',
 '李锟-合川专卖店-李灵健13399879976',
 '零散销售',
 '李锟-南川-张显红13896616619',
 '李锟-潼南批发-李秀刚13350328937',
 '李锟-南川县城专卖店-杨淳之13609463539',
 '苏宁结算',
 '零散销售',
 '万进腾商贸有限公司',
 '王海梦-荣昌区荣隆镇专柜-夏兴国15823312458',
 '零散销售',
 '王海梦-大足龙水-伍敏15823806199',
 '国美结算',
 '李锟-铜梁柳朋-15923286764',
 '零散销售',
 '李锟-铜梁柳朋-15923286764',
 '万进腾商贸有限公司',
 '王海梦-大足区龙水镇专卖店-赵春13896155668',
 '零散销售',
 '苏宁结算',
 '苏宁结算',
 '国美结算',
 '王海梦-石柱秦庭会18996922419',
 '李锟-铜梁柳朋-15923286764',
 '零散销售',
 '李锟-潼南批发-李秀刚13350328937',
 '国美结算',
 '苏宁结算',
 '零散销售',
 '零散销售',
 '王海梦-石柱秦庭会18996922419',
 '罗阳天-秀山县专卖店-王俊凯18896183679',
 '李锟-铜梁柳朋-15923286764',
 '李锟-万盛区专卖店-付家华18523547469',
 '国美结算',
 '王海梦-界石曙光工业园-赵相虎13808301493',
 '零散销售',
 '苏宁结算',
 '王海梦-大足万古-欧丽梅13308326062',
 '零散销售',
 '罗阳天-彭水县专卖店-曾林周13594936789',
 '王海梦-大足区龙水镇专卖店-赵春13896155668',
 '万进腾商贸有限公司',
 '罗阳天-散货',
 '李锟-万盛区专卖店-傅建春13370751538',
 '李锟-丰都高仕林-18581407666',
 '李锟-万盛区专卖店-傅建春13370751538',
 '零散销售',
 '李锟-铜梁柳朋-15923286764',
 '李锟-万盛区专卖店-傅建春13370751538',
 '王海梦-永川区专卖店-陶波18623611600',
 '苏宁结算',
 '李锟-铜梁柳朋-15923286764',
 '李锟-万盛区专卖店-傅建春13370751538',
 '李锟-铜梁柳朋-15923286764',
 '李锟-万盛区专卖店-傅建春13370751538',
 '王海梦-石柱秦庭会18996922419',
 '王海梦-大足龙水-伍敏15823806199',
 '万进腾商贸有限公司',
 '苏宁结算',
 '王海梦-大足青少年宫万家乐专卖店-田飞燕13527589490',
 '李锟-潼南-刘晓飞13983654568',
 '苏宁结算',
 '李锟-长寿美大集成灶-雷相芳13114045376',
 '李锟-铜梁柳朋-15923286764',
 '零散销售',
 '李锟-长寿美大集成灶-雷相芳13114045376',
 '零散销售',
 '零散销售',
 '万进腾商贸有限公司',
 '罗阳天-武隆区专卖店-窦晓梅13228536396',
 '罗阳天-武隆徐开福15736571888',
 '李锟-丰都高仕林-18581407666',
 '国美结算',
 '罗阳天-武隆徐开福15736571888',
 '王海梦-长生桥专卖店-张期文13696471299',
 '李锟-长寿区-黄友明13368313748',
 '王海梦-江津区小官山专卖店-聂小敏13648366797',
 '王海梦-白市驿专卖店-李政15213145597',
 '王海梦-鱼洞中干道专卖店-杜贞兰15723344238',
 '李锟-散货',
 '王海梦-石柱秦庭会18996922419',
 '万进腾商贸有限公司',
 '王海梦-渝北区回兴街道专卖店-张皓月15523325256',
 '王海梦-渝中区新华路批发商-郑平13320203551',
 '苏宁结算',
 '王海梦-渝中区新华路批发商-郑平13320203551',
 '王海梦-大足龙水-伍敏15823806199',
 '李锟-涪陵专卖店-欧华13896660130',
 '国美结算',
 '王海梦-主城九区散货',
 '万进腾商贸有限公司',
 '李锟-铜梁柳朋-15923286764',
 '李锟-长寿桃花大道-余小群15123037348',
 '李锟-散货',
 '苏宁结算',
 '国美结算',
 '李锟-潼南批发-李秀刚13350328937',
 '国美结算',
 '李锟-合川专卖店-李灵健13399879976',
 '苏宁结算',
 '李锟-长寿桃花大道-余小群15123037348',
 '王海梦-主城九区散货',
 '李锟-铜梁柳朋-15923286764',
 '李锟-潼南批发-李秀刚13350328937',
 '王海梦-璧山区专卖店-丁应发18996039366',
 '王海梦-石柱秦庭会18996922419',
 '王海梦-大足青少年宫万家乐专卖店-田飞燕13527589490',
 '李锟-涪陵清溪镇-喻述琳13996813739',
 '国美结算',
 '王海梦-主城九区散货',
 '商渠及其他客户',
 '苏宁结算',
 '苏宁结算',
 '王海梦-永川区专卖店-陶波18623611600',
 '李锟-长寿美大集成灶-雷相芳13114045376',
 '万进腾商贸有限公司',
 '王海梦-鱼洞中干道专卖店-杜贞兰15723344238',
 '李锟-涪陵专卖店-欧华13896660130',
 '李锟-散货',
 '李锟-长寿-陈波13983364973',
 '王海梦-大足龙水-伍敏15823806199',
 '罗阳天-黔江区专卖店-吴仕泉13996970436',
 '李锟-潼南批发-李秀刚13350328937',
 '李锟-南川-张显红13896616619',
 '苏宁结算',
 '李锟-潼南专卖店-贺健全13983615883',
 '罗阳天-黔江-白江13308272849',
 '王海梦-荣昌区荣隆镇专柜-夏兴国15823312458',
 '罗阳天-彭水县专卖店-曾林周13594936789',
 '王海梦-主城九区散货',
 '王海梦-大足龙水-伍敏15823806199',
 '王海梦-永川区专卖店-陶波18623611600',
 '国美结算',
 '王海梦-石柱秦庭会18996922419',
 '苏宁结算',
 '罗阳天-武隆何梅17783326220',
 '王海梦-石柱秦庭会18996922419',
 '万进腾商贸有限公司',
 '王海梦-渝北区回兴街道专卖店-张皓月15523325256',
 '国美结算',
 '李锟-长寿美大集成灶-雷相芳13114045376',
 '零散销售',
 '李锟-綦江九龙大道帅康厨卫-胡小松15823568881',
 '王海梦-鱼洞中干道专卖店-杜贞兰15723344238',
 '苏宁结算',
 '零散销售',
 '李锟-涪陵专卖店-欧华13896660130',
 '王海梦-渝中区新华路批发商-郑平13320203551',
 '王海梦-荣昌区-王梅13996069608',
 '零散销售',
 '万进腾商贸有限公司',
 '万进腾商贸有限公司',
 '罗阳天-黔江区专卖店-吴仕泉13996970436',
 '罗阳天-彭水县专卖店-曾林周13594936789',
 '苏宁结算',
 '李锟-铜梁柳朋-15923286764',
 '罗阳天-彭水县专卖店-曾林周13594936789',
 '王海梦-大足龙水-伍敏15823806199',
 '李锟-南川区水江镇专卖店-郑志刚15023505238',
 '李锟-潼南批发-李秀刚13350328937',
 '罗阳天-彭水县专卖店-曾林周13594936789',
 '王海梦-大足青少年宫万家乐专卖店-田飞燕13527589490',
 '罗阳天-黔江区专卖店-吴仕泉13996970436',
 '李锟-綦江专卖店-贺洪伟13594606456',
 '李锟-丰都高仕林-18581407666',
 '国美结算',
 '王海梦-界石曙光工业园-赵相虎13808301493',
 '国美结算',
 '万进腾商贸有限公司',
 '王海梦-大足龙腾建材市场专卖店-刘平18290408883',
 '王海梦-江津区小官山专卖店-聂小敏13648366797',
 '苏宁结算',
 '王海梦-江津区小官山专卖店-聂小敏13648366797',
 '万进腾商贸有限公司',
 '王海梦-石柱秦庭会18996922419',
 '王海梦-巴南区华南城-牟华13308336922',
 '王海梦-石柱秦庭会18996922419',
 '王海梦-石柱秦庭会18996922419',
 '国美结算',
 '王海梦-荣昌区-王梅13996069608',
 '李锟-丰都县专卖店-苏正勇17320389351',
 '李锟-綦江专卖店-贺洪伟13594606456',
 '国美结算',
 '王海梦-巴南区华南城-牟华13308336922',
 '零散销售',
 '零散销售',
 '李锟-铜梁柳朋-15923286764',
 '李锟-南川火星人陈忠祥13368044900',
 '王海梦-江津区小官山专卖店-聂小敏13648366797',
 '国美结算',
 '王海梦-石柱秦庭会18996922419',
 '零散销售',
 '罗阳天-武隆区专卖店-窦晓梅13228536396',
 '王海梦-大足龙腾建材市场专卖店-刘平18290408883',
 '罗阳天-商渠客户-谷先生13618311605',
 '王海梦-大足-陈莉13983273966',
 '创新渠道户',
 '创新渠道户',
 '零售-赖品利',
 '零售-赖品利',
 '华轻已售未提',
 '国美万盛-韩莉18324162829',
 '商渠-地暖零售客户',
 '商渠-地暖零售客户',
 '居然之家大渡口店',
 '商渠-地暖零售客户',

     '华轻已售未提',
    '李锟-合川专卖店李灵健13399879976',
 '罗阳天-黔江区专卖店吴仕泉13996970436',
 '李锟-涪陵区白涛镇专卖店高勇13983330380',
 '李锟-潼南批发李秀刚13350328937',
 '李锟-合川专卖店李灵健13399879976',
 '罗阳天-酉阳县专卖店黄忠平13996953429',
 '王海梦-璧山区专卖店丁应发18996039366',
 '罗阳天-黔江白江13308272849',
 '李锟-南川区水江镇专卖店郑志刚15023505238',
 '罗阳天-酉阳县专卖店黄忠平13996953429',
 '王海梦-江津区小官山专卖店聂小敏13648366797',
 '李锟-铜梁柳朋15923286764',
 '王海梦-江津区小官山专卖店聂小敏13648366797',
 '王波-大足龙腾建材市场专卖店刘平18290408883',
 '李锟-涪陵专卖店欧华13896660130',
 '王波-大足龙腾建材市场专卖店刘平18290408883',
 '李锟-涪陵专卖店欧华13896660130',
 '李锟-铜梁柳朋15923286764',
 '李锟-南川张显红13896616619',
 
 '李锟-南川专卖店王俊利13908380118',
 '李锟-涪陵专卖店欧华13896660130',
 '罗阳天-武隆区专卖店窦晓梅13228536396',
 '王海梦-鱼洞中干道专卖店杜贞兰15723344238',
 '王海梦-江津区小官山专卖店聂小敏13648366797',
 '李锟-潼南批发李秀刚13350328937',
 '李锟-铜梁柳朋15923286764',
 '罗阳天-武隆区专卖店窦晓梅13228536396',
 '罗阳天-黔江区专卖店吴仕泉13996970436',
 '李锟-涪陵区白涛镇专卖店高勇13983330380',
 '李锟-潼南批发李秀刚13350328937',
 '王海梦-鱼洞中干道专卖店杜贞兰15723344238',
 '罗阳天-酉阳县专卖店黄忠平13996953429',
 '王海梦-江津区小官山专卖店聂小敏13648366797',
 '王海梦-江津区小官山专卖店聂小敏13648366797',
 '李锟-长寿桃花大道余小群15123037348',
 '罗阳天-黔江区专卖店吴仕泉13996970436',
 '李锟-涪陵专卖店欧华13896660130',
 '李锟-南川专卖店王俊利13908380118',
 '李锟-合川专卖店李灵健13399879976',
 '王海梦-永川区专卖店陶波18623611600',
 '王波-大足龙腾建材市场专卖店刘平18290408883',
 '王海梦-大足双桥王波13509427312',
 '王波-渝中区新华路批发商郑平13320203551',
 '罗阳天-彭水县刘小文15023985889',
 '王海梦-永川区专卖店陶波18623611600',
 '王海梦-白市驿专卖店李政15213145597',
 '王海梦-江津区小官山专卖店聂小敏13648366797',
 '李锟-涪陵专卖店欧华13896660130',
 '王波-渝中区新华路批发商郑平13320203551',
 '李锟-涪陵区白涛镇专卖店高勇13983330380',
 '王海梦-大足区龙水镇专卖店赵春13896155668',
 '王海梦-大足龙水伍敏15823806199',
 '王海梦-永川区专卖店陶波18623611600',
 '王波-大足龙腾建材市场专卖店刘平18290408883',
 '王海梦-江津区小官山专卖店聂小敏13648366797',
 '王海梦-大足龙水伍敏15823806199',
 '王海梦-永川区专卖店陶波18623611600',
 '王波-大足龙腾建材市场专卖店刘平18290408883',
 '王海梦-永川区专卖店陶波18623611600',
 '王海梦-大足龙水伍敏15823806199',
 '王海梦-九龙坡区西彭镇专卖店徐老板13883422468',
 '李锟-铜梁柳朋15923286764',
 '王波-渝中区新华路批发商郑平13320203551',
 '王海梦-白市驿专卖店李政15213145597',
 '李锟-南川区水江镇专卖店郑志刚15023505238',
 '罗阳天-武隆区专卖店窦晓梅13228536396',
 '王波-大足龙腾建材市场专卖店刘平18290408883',
 '王海梦-江津区小官山专卖店聂小敏13648366797',
 '罗阳天-彭水县刘小文15023985889',
 '李锟-铜梁柳朋15923286764',
 '王波-九龙坡区曾家镇专卖店唐勇13983220787',
 '罗阳天-黔江白江13308272849',
 '王波-九龙坡区曾家镇专卖店唐勇13983220787',
 '王海梦-永川区专卖店陶波18623611600',
 '罗阳天-秀山县专卖店王俊凯18896183679',
 '王海梦-大足龙水伍敏15823806199',
 '李锟-潼南批发李秀刚13350328937',
 '李锟-铜梁柳朋15923286764',
 '李锟-丰都高仕林18581407666',
 '王海梦-鱼洞中干道专卖店杜贞兰15723344238',
 '王海梦-鱼洞中干道专卖店杜贞兰15723344238',
 '李锟-涪陵清溪镇喻述琳13996813739',
 '王海梦-永川区专卖店陶波18623611600',
 '王海梦-鱼洞中干道专卖店杜贞兰15723344238',
 '李锟-合川专卖店李灵健13399879976',
 '王海梦-白市驿专卖店李政15213145597',
 '王海梦-鱼洞中干道专卖店杜贞兰15723344238',
 '李锟-铜梁柳朋15923286764',
 '李锟-潼南批发李秀刚13350328937',
 '王海梦-界石曙光工业园赵相虎13808301493',
 '王波-九龙坡区曾家镇专卖店唐勇13983220787',
 '李锟-潼南批发李秀刚13350328937',
 '李锟-涪陵区白涛镇专卖店高勇13983330380',
 '王海梦-白市驿专卖店李政15213145597',
 '李锟-铜梁柳朋15923286764',
 '王海梦-白市驿专卖店李政15213145597',
 '王海梦-白市驿专卖店李政15213145597',
 '王海梦-白市驿专卖店李政15213145597',
 '王波-九龙坡区曾家镇专卖店唐勇13983220787',
 '王海梦-白市驿专卖店李政15213145597',
 '李锟-铜梁柳朋15923286764',
 '王海梦-大足双桥王波13509427312',
 '王波-渝中区新华路批发商郑平13320203551',
 '王波-大足龙腾建材市场专卖店刘平18290408883',
 '王海梦-江津区小官山专卖店聂小敏13648366797',
 '王海梦-界石曙光工业园赵相虎13808301493',
 '李锟-涪陵区白涛镇专卖店高勇13983330380',
 '王海梦-白市驿专卖店李政15213145597',
 '李锟-丰都县专卖店苏正勇17320389351',
 '王海梦-白市驿专卖店李政15213145597']

In [25]:
# 制定出标准价
data_circulate['标准价'] = 0

# 筛选出对应型号，并把对应的标准价赋值。 
data_circulate.loc[(data_circulate['单据类型'].str.contains('万家乐电热水器D60-F1')),'标准价'] = 520

data_circulate.loc[(data_circulate['单据类型'].str.contains('万家乐电热水器D80-F1')),'标准价'] = 620

data_circulate.loc[(data_circulate['单据类型'].str.contains('万家乐电热水器D60-GHF')),'标准价'] = 620

data_circulate.loc[(data_circulate['单据类型'].str.contains('万家乐电热水器D80-GHF')),'标准价'] = 720

# 把老客户的提货数据，给去掉。 
data_circulate = data_circulate.loc[~(data_circulate['客户名称'].isin(x))]


data_circulate['价差'] = data_circulate['销售价']-data_circulate['标准价']


# 提成标准
def ti_cheng(x):
    if x < 0:
        return 10
    elif x == 0:
        return 20
    else:
        return 0.5*x+20

data_circulate['价差'] = data_circulate['价差'].apply(ti_cheng)



data_circulate= data_circulate[['业务','单据类型', '客户名称', '数量', '销售价','标准价', '价差'
                                ]]
data_circulate.columns = ['业务','单据类型', '客户名称', '数量', '销售价','标准价', 
                                '单台提成']

data_circulate['小计提成'] = data_circulate['数量'] * data_circulate['单台提成']



data_circulate[['数量','小计提成']] = data_circulate[['数量','小计提成']].astype('float')

data_circulate.groupby('业务').sum()[['数量','小计提成']]

/home/bobo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/bobo/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,数量,小计提成
业务,,
李锟,21.0,835.0
王海梦,264.0,16306.5
罗阳天,63.0,2555.0
重商新结算,5.0,1609.5


In [26]:
# 需要加上时间维度吗？比如，只测算11月份

data_circulate['2019-11']#.groupby('业务').sum()[['数量','小计提成']]


,业务,单据类型,客户名称,数量,销售价,标准价,单台提成,小计提成
单据日期,,,,,,,,
2019-11-03,李锟,万家乐电热水器D60-F1,李锟-代华强-涪陵余孝锋15213697770,10.0,520,520,20.0,200.0
2019-11-23,王海梦,万家乐电热水器D60-F1,王海梦-主城-鱼洞中干道专卖店杜贞兰15723344238,1.0,660,520,90.0,90.0
2019-11-24,王海梦,万家乐电热水器D80-F1,王海梦-主城-九区散货,1.0,800,620,110.0,110.0
2019-11-27,王海梦,万家乐电热水器D80-GHF(D),王海梦-主城-渝中区新华路批发商郑平13320203551,20.0,750,720,35.0,700.0
2019-11-28,王海梦,万家乐电热水器D80-GHF(D),王海梦-主城-白市驿专卖店李政15213145597,1.0,800,720,60.0,60.0
2019-11-03,王海梦,万家乐电热水器D60-GHF(D),王海梦-主城-白市驿专卖店李政15213145597,3.0,660,620,40.0,120.0
2019-11-14,王海梦,万家乐电热水器D60-GHF(D),王海梦-主城-九龙坡区曾家镇专卖店唐勇13983220787,5.0,650,620,35.0,175.0
2019-11-19,王海梦,万家乐电热水器D60-GHF(D),王海梦-主城-白市驿专卖店李政15213145597,2.0,700,620,60.0,120.0
2019-11-27,王海梦,万家乐电热水器D60-GHF(D),王海梦-大足双桥王海梦13509427312,1.0,660,620,40.0,40.0
